Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------------------------------
# Fetch Debt Data
# ---------------------------------------------------
BASE_URL = "https://api.worldbank.org/v2/country/all/indicator/DT.DOD.DPPG.CD"
PARAMS = {"format": "json", "date": "2024", "per_page": "5000"}

response = requests.get(BASE_URL, params=PARAMS)
data = response.json()

if len(data) < 2:
    raise Exception("No data returned from API")

records = data[1]

rows = [
    {"country": r["country"]["value"], "debt_usd": float(r["value"])}
    for r in records if r["value"] is not None
]
df = pd.DataFrame(rows)
df = df[df["debt_usd"] > 0].sort_values("debt_usd", ascending=False).reset_index(drop=True)

# ---------------------------------------------------
# Plot Horizontal Bar Chart
# ---------------------------------------------------

num_countries = len(df)
fig_height = max(12, num_countries * 0.25)  # scale figure height dynamically
plt.figure(figsize=(14, fig_height))

bars = plt.barh(df["country"], df["debt_usd"], color="steelblue")
plt.gca().invert_yaxis()

plt.title("Country External Debt Stocks (Total, Current US$)", fontsize=14)
plt.xlabel("Debt in USD (Current)", fontsize=12)

# Add combined rank + debt label outside the bar
max_debt = df["debt_usd"].max()
for i, (country, val) in enumerate(zip(df["country"], df["debt_usd"])):
    label = f"rank {i+1}. ${val:,.0f}"
    plt.text(val + max_debt * 0.005, bars[i].get_y() + bars[i].get_height()/2,
             label, va="center", fontsize=8)

plt.tight_layout()
plt.show()
